# Biotest - List

In [12]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

#OCR
from wand.image import Image
import pyocr
import pyocr.builders
from PIL import Image, ImageFilter

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *



import re

## Export Images    

In [20]:
def read_pipe_separated_str(str_input: str, **kwargs) -> pd.DataFrame:
    #str_input = str_input.replace("\n", "\\n")
    """Read a Pandas object from a pipe-separated table contained within a string.

    Example:
        | int_score | ext_score | eligible |
        |           | 701       | True     |
        | 221.3     | 0         | False    |
        |           | 576       | True     |
        | 300       | 600       | True     |

    The leading and trailing pipes are optional, but if one is present, so must be the other.

    `kwargs` are passed to `read_csv`. They must not include `sep`.

    In PyCharm, the "Pipe Table Formatter" plugin has a "Format" feature that can be used to neatly format a table.
    """
    # Ref: https://stackoverflow.com/a/46471952/
    substitutions = [
        ('^ *', ''),  # Remove leading spaces
        (' *$', ''),  # Remove trailing spaces
        (r' *\| *', '|'),  # Remove spaces between columns
    ]
    if all(line.lstrip().startswith('|') and line.rstrip().endswith('|') for line in str_input.strip().split('\n')):
        substitutions.extend([
            (r'^\|', ''),  # Remove redundant leading delimiter
            (r'\|$', ''),  # Remove redundant trailing delimiter
        ])
    for pattern, replacement in substitutions:
        str_input = re.sub(pattern, replacement, str_input, flags=re.MULTILINE)
    
    print(str_input)
    #return pd.read_csv(pd.compat.StringIO(str_input), sep='|', **kwargs)

In [3]:

with Image(filename='test.pdf', resolution=300) as img:
    img.compression_quality = 99
    img.save(filename='out.jpg')

In [9]:
img = Image.open('out.jpg')

In [21]:
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

tool = tools[0]
langs = tool.get_available_languages()
langs = 'eng+deu'

txt = tool.image_to_string(
    img,
    #img,
    lang=langs,
    builder=pyocr.builders.TextBuilder()
)


In [23]:
txt

"Geldwerte Leistungen im Zusammenhang Dienstleistungs- und\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n     \n  \n     \n    \n    \n   \n    \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n   \n \n\n  \n \n\n  \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n   \n  \n\n \n\n \n\n   \n \n\n    \n \n\n \n\n    \n \n\n \n\n \n\n \n\n \n\n   \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nmit Veranstaltungen Beratungshonorare\nFinanzielle oder Unterstützung von\nmaterielle Spenden aa GESAMT\nsowie Förderungen Organisationen oder .\nvon diesen mit der Tagungs- und . Reise- und Hereisrs\nDurchführung der Teilnahmegebühren | Übernachtungskosten\nVeranstaltung\nName Land| PLZ Ort

In [22]:
df = read_pipe_separated_str(txt)

Geldwerte Leistungen im Zusammenhang Dienstleistungs- und


















































































































































































































































mit Veranstaltungen Beratungshonorare
Finanzielle oder Unterstützung von
materielle Spenden aa GESAMT
sowie Förderungen Organisationen oder .
von diesen mit der Tagungs- und . Reise- und Hereisrs
Durchführung der Teilnahmegebühren|Übernachtungskosten
Veranstaltung
Name Land|PLZ Ort Strasse beauftragte Dritte
Dr. med. Bodo Grahlke CH|8134 Adliswil Kilchbergstrasse 19 355.63 CHF 1'753.50 CHF 2'109.13 CHF
Prof. Dr. Felix Stickel|CH} 3074 Muri Haldenweg H 304.00 CHF 1'400.00 CHF 1'704.00 CHF
Prof. Dr. Lars E. French CH; 8006 Zürich Rämistrasse 100 278.45 CHF} 1'150.60 CHF 1'429.05 CHF
Dr. Joachim Mertens CH; 8006 Zürich Rämistrasse 100 433.15 CHF 433.15 CHF
Dr. J. Vionnet CH|1011 Lausanne Rue de

## Import PDF

In [ ]:
# Read pdf into DataFrame
df = tabula.read_pdf("out.pdf", pages='all', lattice=False)

In [ ]:
df.head()

## Format Table

In [ ]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2']

#Remove "HCP" & "HCO"
df_export.loc[df_export['empty'].str.contains("HCP", na=False), 'empty'] = np.NaN
df_export.loc[df_export['empty'].str.contains("HCO", na=False), 'empty'] = np.NaN
df_export.loc[df_export['empty'].str.contains("R&D", na=False), 'empty'] = np.NaN

#Shift all where empty not empty
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Copy empty1 to total, if not empty
df_export.loc[df_export['empty1'].notna(), 'total'] = df_export['empty1']

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty
df_export.drop(columns=['empty', 'empty1', 'empty2'], inplace=True)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Set Type
df_export = set_type_by_alphabetical_order(df_export)


#Remove Carrination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'biogen')


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')